In [2]:
#imports
import pandas as pd
import os
import sqlite3 as sq3
import datetime as date
import json
import uuid

In [18]:
#prepare data and put into correct format for LostIslandBuildings
def prepareData():

    df = pd.read_csv('LostIslandBuildings.csv')

    #Rename Color / Area column to be Color
    df.rename(columns={'Color / Area': 'Color'}, inplace=True)

    #Separate Coordinates into X-Coord and Y-Coord
    df[['X-Coord', 'Y-Coord']] = df['Coordinates'].str.split(' ', expand=True)
    df = df.drop('Coordinates', axis=1)
    df['X-Coord'] = df['X-Coord'].str[2:]
    df['Y-Coord'] = df['Y-Coord'].str[2:]

    #Separate Buff Percent into categories
    buffCategories = ['HP Guardian', 'HP Shooter', 'HP Carrier', 'ATK Guardian', 'ATK Shooter', 'ATK Carrier', 'DEF Guardian', 'DEF Shooter', 'DEF Carrier']

    for i in range(len(buffCategories)):
        df[buffCategories[i]] = df['Buff Percent'].apply(lambda x: x.split(' ')[0] if buffCategories[i] in x else 0)

    #Separate Buff Percent into categories for Wonders
    wonderCondition = df['Alliance Influence'] == '25,000'
    wonderRowsOnly = df.loc[wonderCondition, :]
    wonderRowBuffs = ['3.6%', '3.6%', '3.6%', '9%', '9%', '9%', '9%', '9%', '9%']
    for i in range(len(buffCategories)):
        df.loc[wonderCondition, buffCategories[i]] = wonderRowBuffs[i]
    df.loc[wonderCondition, buffCategories] = wonderRowsOnly[buffCategories]

    df = df.drop('Buff Percent', axis=1)

    #Separate Soldier Production into categories
    productionCategories = ['Guardian', 'Shooter', 'Carrier']

    for i in range(len(productionCategories)):
        df[productionCategories[i] + ' Output'] = df['Soldier Production'].apply(lambda x: x.split(' ')[0] if productionCategories[i] in x else 0)

    #Separate Soldier Production for Wonders
    wonderRowProduction = ['44/h', '44/h', '44/h']
    productionCategoriesRenamed = ['Guardian Output', 'Shooter Output', 'Carrier Output']
    for i in range(len(productionCategoriesRenamed)):
        wonderRowsOnly[productionCategoriesRenamed[i]] = wonderRowProduction[i]
    df.loc[wonderCondition, productionCategoriesRenamed] = wonderRowsOnly[productionCategoriesRenamed]

    df = df.drop('Soldier Production', axis=1)

    #Rename Buildings to Building
    df.rename(columns={'Buildings': 'Building'}, inplace=True)

    #Formatting
    df = df.apply(lambda x: x.str.replace('%', '', regex=True))
    df = df.apply(lambda x: x.str.replace('/h', '', regex=True))
    df = df.apply(lambda x: x.str.replace(',', '', regex=True))
    df = df.fillna(0)

    df['Alliance Influence'] = df['Alliance Influence'].astype('int')
    for i in range(len(buffCategories)):
        df[buffCategories[i]] = df[buffCategories[i]].astype(float)
    for i in range(len(productionCategoriesRenamed)):
        df[productionCategoriesRenamed[i]] = df[productionCategoriesRenamed[i]].astype('int')

    df.to_csv('UpdatedLostIslandBuildings.csv', index=False)

In [19]:
#Convert csv file to SQLite database
def convertToSQLiteDB():
    updatedDf = pd.read_csv('UpdatedLostIslandBuildings.csv')
    conn = sq3.connect('LostIslandBuildings.db')
    updatedDf.to_sql('Buildings', conn, if_exists='replace', index=False)
    conn.close()

In [13]:
#Create empty SQLite DB with only the headers initialized for a new Lost Island and setup textfile with alliance names
def initializeNewIsland(pinkName="", blueName="", brownName="", greenName="", purpleName="", yellowName="", tealName="", orangeName=""):
    current_date = date.datetime.now().strftime("%Y-%m-%d")
    save_path = "Events/" + current_date
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    #Connect to the sq3 db
    conn = sq3.connect(save_path + "/LostIslandAllianceBuildings.db")
    #Create cursor obj to exec SQL commands
    cursor = conn.cursor()

    #Table schema
    table_schema = """
    CREATE TABLE IF NOT EXISTS alliance_buildings (
        id INTEGER PRIMARY KEY,
        Alliance TEXT,
        Building TEXT,
        Color TEXT,
        "Alliance Influence" INTEGER,
        "X-Coord" INTEGER,
        "Y-Coord" INTEGER,
        "HP Guardian" FLOAT,
        "HP Shooter" FLOAT,
        "HP Carrier" FLOAT,
        "ATK Guardian" FLOAT,
        "ATK Shooter" FLOAT,
        "ATK Carrier" FLOAT,
        "DEF Guardian" FLOAT,
        "DEF Shooter" FLOAT,
        "DEF Carrier" FLOAT,
        "Guardian Output" INTEGER,
        "Shooter Output" INTEGER,
        "Carrier Output" INTEGER
    );"""
    cursor.execute(table_schema)
    conn.commit()
    conn.close()

    #Create config file
    islandConfig = save_path + "/config.json"

    islandConfigData = {
        "date": current_date,
        "alliances": {
            "pink": pinkName,
            "blue": blueName,
            "brown": brownName,
            "green": greenName,
            "purple": purpleName,
            "yellow": yellowName,
            "teal": tealName,
            "orange": orangeName
        }
    }

    with open(islandConfig, 'w') as configFile:
        json.dump(islandConfigData, configFile)
        
#testing
#initializeNewIsland("EOA", "", "LAO", "SEE", "SeN", "", "TAO")

In [12]:
#Add Building to Alliance Buildings given X-Y coordinates and the occupying alliance
def addBuilding(date, allianceName, xCoord, yCoord):
    buildingPath = "LostIslandBuildings.db"
    allianceBuildingPath = "Events/" + date + "/LostIslandAllianceBuildings.db"

    #Fetch building information
    buildingConn = sq3.connect(buildingPath)
    cursor = buildingConn.cursor()
    cursor.execute('SELECT * FROM Buildings WHERE "X-Coord" = ? AND "Y-Coord" = ?', (xCoord, yCoord)) 
    building = cursor.fetchall()
    cursor.close()
    buildingConn.close()

    #Format data
    buildingName = building[0][0]
    color = building[0][1]
    influence = int(building[0][2])
    hpGuard = float(building[0][5])
    hpShoot = float(building[0][6])
    hpCarr = float(building[0][7])
    atkGuard = float(building[0][8])
    atkShoot = float(building[0][9])
    atkCarr = float(building[0][10])
    defGuard = float(building[0][11])
    defShoot = float(building[0][12])
    defCarr = float(building[0][13])
    guardOut = int(building[0][14])
    shootOut = int(building[0][15])
    carrOut = int(building[0][16])

    #Add to alliance building
    allianceBuildingConn = sq3.connect(allianceBuildingPath)
    cursor1 = allianceBuildingConn.cursor()
    #Check if already exists
    cursor1.execute('SELECT Alliance FROM alliance_buildings WHERE "X-Coord" = ? AND "Y-Coord" = ?', (xCoord, yCoord))
    existingBuildingRow = cursor1.fetchone()
    if not existingBuildingRow:
        cursor1.execute('INSERT INTO alliance_buildings (Alliance, Building, Color, "Alliance Influence", "X-Coord", "Y-Coord", "HP Guardian", "HP Shooter", "HP Carrier", "ATK Guardian", "ATK Shooter", "ATK Carrier", "DEF Guardian", "DEF Shooter", "DEF Carrier", "Guardian Output", "Shooter Output", "Carrier Output") VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', (allianceName, buildingName, color, influence, xCoord, yCoord, hpGuard, hpShoot, hpCarr, atkGuard, atkShoot, atkCarr, defGuard, defShoot, defCarr, guardOut, shootOut, carrOut))
    #Case: check if exists under different alliance
    elif existingBuildingRow != allianceName:
         cursor1.execute('UPDATE alliance_buildings SET Alliance = ? WHERE "X-Coord" = ? AND "Y-Coord" = ?', (allianceName, xCoord, yCoord))
    allianceBuildingConn.commit()
    allianceBuildingConn.close()

#testing
#addBuilding("2023-10-21", "ARA", 543, 576)
#addBuilding("2023-10-21", "ARA", 543, 576)
#addBuilding("2023-10-21", "ARA", 333, 1315)
#addBuilding("2023-10-21", "ARA", 314, 1074)

In [7]:
#delete a specific ID from an Alliance's buildings
def deleteIDFromAllianceBuildings(date, allianceName, xCoord, yCoord):
    allianceBuildingsPath = 'Events/' + date + '/LostIslandAllianceBuildings.db'
    allianceBuildingsConn = sq3.connect(allianceBuildingsPath)
    cursor = allianceBuildingsConn.cursor()
    cursor.execute('DELETE from alliance_buildings WHERE alliance = ? AND "X-Coord" = ? AND "Y-Coord" = ?', (allianceName, xCoord, yCoord))
    allianceBuildingsConn.commit()
    allianceBuildingsConn.close()

#testing
#deleteIDFromAllianceBuildings("2023-10-21", "EOA", 543, 576)

In [47]:
#Get totals for buffs
def getBuffTotalsForAlliance(date, allianceName):
    allianceBuildingsPath = 'Events/' + date + '/LostIslandAllianceBuildings.db'
    allianceBuildingsConn = sq3.connect(allianceBuildingsPath)
    cursor = allianceBuildingsConn.cursor()
    cursor.execute('SELECT SUM("Alliance Influence"), SUM("HP Guardian"), SUM("HP Shooter"), SUM("HP Carrier"), SUM("ATK Guardian"), SUM("ATK Shooter"), SUM("ATK Carrier"), SUM("DEF Guardian"), SUM("DEF Shooter"), SUM("DEF Carrier"), SUM("Guardian Output"), SUM("Shooter Output"), SUM("Carrier Output") from alliance_buildings WHERE Alliance = ?', (allianceName,))
    allSums = cursor.fetchone()
    allianceInfluenceSum = allSums[0]
    hpGuardianSum = allSums[1]
    hpShooterSum = allSums[2]
    hpCarrierSum = allSums[3]
    atkGuardianSum = allSums[4]
    atkShooterSum = allSums[5]
    atkCarrierSum = allSums[6]
    defGuardianSum = allSums[7]
    defShooterSum = allSums[8]
    defCarrierSum = allSums[9]
    guardianOutSum = allSums[10]
    shooterOutSum = allSums[11]
    carrierOutSum = allSums[12]

    outputJson = {
        "alliance": allianceName,
        "alliance influence sum": allianceInfluenceSum if allianceInfluenceSum is not None else 0.0,
        "hp guardian sum": hpGuardianSum if hpGuardianSum is not None else 0.0,
        "hp shooter sum": hpShooterSum if hpShooterSum is not None else 0.0,
        "hp carrier sum": hpCarrierSum if hpCarrierSum is not None else 0.0,
        "atk guardian sum": atkGuardianSum if atkGuardianSum is not None else 0.0,
        "atk shooter sum": atkShooterSum if atkShooterSum is not None else 0.0,
        "atk carrier sum": atkCarrierSum if atkCarrierSum is not None else 0.0,
        "def guardian sum": defGuardianSum if defGuardianSum is not None else 0.0,
        "def shooter sum": defShooterSum if defShooterSum is not None else 0.0,
        "def carrier sum": defCarrierSum if defCarrierSum is not None else 0.0,
        "guardian output sum": guardianOutSum if guardianOutSum is not None else 0,
        "shooter output sum": shooterOutSum if shooterOutSum is not None else 0,
        "carrier output sum": carrierOutSum if carrierOutSum is not None else 0
    }

    return outputJson

#print(getBuffTotalsForAlliance("2023-10-21", "ARA"))

In [53]:
#Get totals for buffs for all alliances in single Lost Island
def getAllBuffTotals(date):
    alliancePath = "Events/" + date + "/config.json"
    allAllianceBuffTotal = {}
    with open(alliancePath, 'r') as jsonFile:
        configData = json.load(jsonFile)
        for color, alliance in configData['alliances'].items():
            #only for colors that are assigned to an alliance
            if alliance != '':
                allAllianceBuffTotal[color] = getBuffTotalsForAlliance(date, alliance)
    return allAllianceBuffTotal
#testing
buffTotals = getAllBuffTotals("2023-10-21")

In [81]:
#Updates which alliance is the highest for each buff
def getAllHighestBuffs(date, buffTotals):
    highestBuffsOutput = {
        "alliance influence": (0, ""),
        "hp guardian sum": (0.0, ""),
        "hp shooter sum": (0.0, ""),
        "hp carrier sum": (0.0, ""),
        "atk guardian sum": (0.0, ""),
        "atk shooter sum": (0.0, ""),
        "atk carrier sum": (0.0, ""),
        "def guardian sum": (0.0, ""),
        "def shooter sum": (0.0, ""),
        "def carrier sum": (0.0, ""), 
        "guardian output sum": (0, ""),
        "shooter output sum": (0, ""),
        "carrier output sum": (0, "")
    }
    for key, value in buffTotals.items():
        if value['alliance influence sum'] > highestBuffsOutput['alliance influence'][0]:
            highestBuffsOutput['alliance influence'] = (value['alliance influence sum'], value['alliance'])
        if value['hp guardian sum'] > highestBuffsOutput['hp guardian sum'][0]:
            highestBuffsOutput['hp guardian sum'] = (value['hp guardian sum'], value['alliance'])
        if value['hp shooter sum'] > highestBuffsOutput['hp shooter sum'][0]:
            highestBuffsOutput['hp shooter sum'] = (value['hp shooter sum'], value['alliance'])
        if value['hp carrier sum'] > highestBuffsOutput['hp carrier sum'][0]:
            highestBuffsOutput['hp carrier sum'] = (value['hp carrier sum'], value['alliance'])
        if value['atk guardian sum'] > highestBuffsOutput['atk guardian sum'][0]:
            highestBuffsOutput['atk guardian sum'] = (value['atk guardian sum'], value['alliance'])
        if value['atk shooter sum'] > highestBuffsOutput['atk shooter sum'][0]:
            highestBuffsOutput['atk shooter sum'] = (value['atk shooter sum'], value['alliance'])
        if value['atk carrier sum'] > highestBuffsOutput['atk carrier sum'][0]:
            highestBuffsOutput['atk carrier sum'] = (value['atk carrier sum'], value['alliance'])
        if value['def guardian sum'] > highestBuffsOutput['def guardian sum'][0]:
            highestBuffsOutput['def guardian sum'] = (value['def guardian sum'], value['alliance'])
        if value['def shooter sum'] > highestBuffsOutput['def shooter sum'][0]:
            highestBuffsOutput['def shooter sum'] = (value['def shooter sum'], value['alliance'])
        if value['def carrier sum'] > highestBuffsOutput['def carrier sum'][0]:
            highestBuffsOutput['def carrier sum'] = (value['def carrier sum'], value['alliance'])
        if value['guardian output sum'] > highestBuffsOutput['guardian output sum'][0]:
            highestBuffsOutput['guardian output sum'] = (value['guardian output sum'], value['alliance'])
        if value['shooter output sum'] > highestBuffsOutput['shooter output sum'][0]:
            highestBuffsOutput['shooter output sum'] = (value['shooter output sum'], value['alliance'])
        if value['carrier output sum'] > highestBuffsOutput['carrier output sum'][0]:
            highestBuffsOutput['carrier output sum'] = (value['carrier output sum'], value['alliance'])
    return highestBuffsOutput
            
#testing
#getAllHighestBuffs("2023-10-21", buffTotals)

In [82]:
#Updates entire output.json file with buff totals and the highest buffs
def updateOutputConfig(date):
    path = "Events/" + date + "/output.json"
    buffTotals = getAllBuffTotals(date)
    highestBuffs = getAllHighestBuffs(date, buffTotals)
    outputJson = {
        "buff totals": buffTotals,
        "highest buffs": highestBuffs
    }
    with open(path, 'w') as jsonFile:
        json.dump(outputJson, jsonFile)
#testing
#updateOutputConfig("2023-10-21")